# **Drive mount/ Common**

In [1]:
from google.colab import drive # import drive from google colab
ROOT = "/content/drive"     # default location for the drive
PROJECT_PATH = ROOT+'/My Drive/Fall2020/My_Capstone/'
drive.mount(ROOT)

Mounted at /content/drive


In [2]:
%cd "{PROJECT_PATH}"
#!git clone -b Pritam "{GIT_PATH}"
%cd "Edge"


/content/drive/My Drive/Fall2020/My_Capstone
/content/drive/My Drive/Fall2020/My_Capstone/Edge


In [3]:
def quantization_eval_results(model_name,train_set,test_set,batch_size,criterion, method='all'):
  results = quantization(model_name, method)
  train_loss_list = []
  train_accuracy_list = []
  test_loss_list = []
  test_accuracy_list = []
  for i in results["model_artifact"]:
    train_loss, train_accuracy = evaluate(model=i, 
                                        test_set = train_set,
                                        batch_size=batch_size, 
                                        criterion=criterion,
                                        ep=0) 
    test_loss, test_accuracy = evaluate(model=i, 
                                        test_set = test_set,
                                        batch_size=batch_size, 
                                        criterion=criterion,
                                        ep=0)  
    train_loss_list.append(train_loss.item())
    test_loss_list.append(test_loss.item())
    train_accuracy_list.append(train_accuracy)
    test_accuracy_list.append(test_accuracy)
    # print("End of training, test loss =  {}, test accuracy = {} \n".format(train_loss, train_accuracy))
    # print("End of training, test loss =  {}, test accuracy = {} \n".format(test_loss, test_accuracy))
  results["train_loss"] = train_loss_list
  results["train_acc"] = train_accuracy_list
  results["test_loss"] = test_loss_list
  results["test_acc"] = test_accuracy_list
  return results

# **Model = ConvNet, Data = CIFAR100 data**
Val Accuracy=46%

In [ ]:
# %%writefile train_cifar.py
import time
import torch
import os
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torchsummary import summary

from model.dnn import DenseNeuralNet
from model.cnn import CNN
from data.churn_data import ChurnDataset
from data.telescope_data import TelescopeDataset
from utils.util_functions import *
from data.mv_data import MVDataset
from tqdm.auto import trange, tqdm
from tqdm import trange
from torch.utils.data import Subset
from sklearn.model_selection import train_test_split


def get_cifar100_dataset(train = True):
  transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5071, 0.4865, 0.4409), std=(0.2673, 0.2564, 0.2762)),
  ])

  transform_test = transforms.Compose([
      transforms.ToTensor(),
      transforms.Normalize(mean=(0.5071, 0.4865, 0.4409), std=(0.2673, 0.2564, 0.2762)),
  ])

  if train == True:
      dataset = datasets.CIFAR100(root = './data', train = train, transform=transform_test, target_transform=None, download=True)
  else:
      dataset = datasets.CIFAR100(root = './data', train = train, transform=transform_test, target_transform=None, download=True)
  
  return dataset


def get_accuracy(logits, labels):
  preds = torch.argmax(logits, axis=1)
  matches = preds == labels
  return (matches.sum(), len(labels))


def evaluate(model, test_set, batch_size, criterion, ep = 0):
  test_loader = torch.utils.data.DataLoader(dataset = test_set, batch_size = batch_size, shuffle=True, num_workers=1)
  test_iterator = tqdm(test_loader, desc = 'Eval Iteration for epoch:'+str(ep+1), ncols = 900)
  device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
  
  model.eval()
  global_step = 0
  total_correct = 0
  total_samples = 0
  total_loss = 0.0
  with torch.no_grad():
    for step, inputs in enumerate(test_iterator):
      global_step +=1
      # if global_step > 500:
      #   break
      x, y = inputs[0].to(device), inputs[1].long().to(device)

      logits = model(x)
      loss = criterion(logits, y)
      correct, samples = get_accuracy(logits, y)
      total_correct +=correct.item()
      total_samples +=samples
      total_loss +=loss
  # print(total_correct, total_samples)
  acc = total_correct / total_samples
  total_loss = total_loss / global_step
  model.train()
  
  return (total_loss, acc)


def train(model, train_set, val_set, test_set , batch_size = 16, learning_rate = 0.03, epochs = 5, eval_steps = 10, skip_train_set = True):
  # logging
  train_log = open("log/cifar_cnn_train.log", "a")
  val_log = open("log/cifar_cnn_val.log", "a")
  test_log = open("log/cifar_cnn_test.log", "a")

  # GPU/CPU use
  device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
  print("Device: ", device)
  model = model.to(device)
  print("Model Summary:")
  summary(model, next(iter(train_set))[0].shape)
  
  # define loss & optimizer
  criterion = nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
  # optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)

  # iterate over epoch
  train_loader = torch.utils.data.DataLoader(dataset= train_set, batch_size=batch_size, shuffle=True, num_workers=1)
  global_step = 0
  for ep in tqdm(range(epochs), desc = ' Epoch Progress:', ncols=900):
    train_iterator = tqdm(train_loader, desc = 'Train Iteration for epoch:'+ str(ep+1), ncols=900)    
    running_loss = 0

    # iterate over batches
    for step, inputs in enumerate(train_iterator):
      model.train()
      global_step +=1
      optimizer.zero_grad()
      # predict, find loss, get grads, update weight
      x, y = inputs[0].to(device), inputs[1].to(device)
      logits = model(x)
      loss = criterion(logits, y)
      loss.backward()
      optimizer.step()
      running_loss+=loss.item()

    # find validation accuracy
    val_loss, val_accuracy = evaluate(model, val_set, batch_size, criterion, ep)
    val_log.write("Epoch = {}, validation loss =  {}, validation accuracy = {} \n".format(ep+1, val_loss, val_accuracy))
    print("Step = %d, validation loss =  %.3f, validation accuracy = %.3f" %(global_step, val_loss, val_accuracy))
    
    # find train accuracy if needed
    if not skip_train_set:
      train_loss , train_accuracy = evaluate(model, train_set, batch_size, criterion, ep)
      train_log.write("Epoch = {}, training loss =  {}, training accuracy = {} \n".format(ep+1, train_loss, train_accuracy))
      print("Step = %d, training loss =  %.3f, training accuracy = %.3f" %(global_step, train_loss, train_accuracy))

  # find test accuracy with final model
  if test_set is not None:  
    test_loss, test_accuracy = evaluate(model, test_set, batch_size, criterion, ep)
    test_log.write("End of training, test loss =  {}, test accuracy = {} \n".format(test_loss, test_accuracy))
    print("End of Training, test loss =  %.3f, test accuracy = %.3f" %(test_loss, test_accuracy))

  # close log files
  train_log.close()
  val_log.close()
  test_log.close()

def main(load_model = False):
  ### config params 
  output_classes = 100
  learning_rate = 0.001
  batch_size = 16
  epochs = 10
  eval_steps = 100
  model_dir = 'model_artifacts'
  model_name = 'cifar_cnn_model.pt'
  criterion = nn.CrossEntropyLoss()
  ####

  train_set, val_set, test_set = None, None, None
  train_set = get_cifar100_dataset(train = True)
  val_set = get_cifar100_dataset(train = False)

  if load_model == True:
    model = torch.load(os.path.join(model_dir, model_name))
    val_loss, val_accuracy = evaluate(model, val_set, batch_size, criterion)
    print("Running evaluation on loaded model, validation loss = %f, validation accuracy = %f"%(val_loss, val_accuracy))

  else:
    model = CNN(output_classes)
    train(model, train_set, val_set, test_set , batch_size = batch_size, learning_rate = learning_rate, epochs = epochs, eval_steps = eval_steps, skip_train_set = False)
    torch.save(model, os.path.join(model_dir, model_name))

  # Evaluate Quantization
  path_result = "data/results/"
  model_results = quantization_eval_results(model_name,train_set=train_set,test_set=val_set,batch_size=batch_size,criterion=criterion, method='all')
  print(model_results)
  model_results.to_csv(path_result + model_name[:-3]+'_v2' +".csv")


if __name__ == "__main__":
  main(load_model = True)



Files already downloaded and verified
Files already downloaded and verified



Running evaluation on loaded model, validation loss = 2.022033, validation accuracy = 0.460300
Results appended for Stochastic Rounding : uniform_range 	 16
Results appended for Stochastic Rounding : uniform_range 	 12
Results appended for Stochastic Rounding : uniform_range 	 10
Results appended for Stochastic Rounding : uniform_range 	 8
Results appended for Stochastic Rounding : uniform_range 	 6
Results appended for Stochastic Rounding : uniform_range 	 4
Results appended for Stochastic Rounding : uniform_range_IQR 	 16
Results appended for Stochastic Rounding : uniform_range_IQR 	 12
Results appended for Stochastic Rounding : uniform_range_IQR 	 10
Results appended for Stochastic Rounding : uniform_range_IQR 	 8
Results appended for Stochastic Rounding : uniform_range_IQR 	 6
Results appended for Stochastic Rounding : uniform_range_IQR 	 4
Results appended for Stochastic Rounding : prior_normal 	 16
Results appended for Stochastic Rounding : prior_normal 	 12
Results appended for

# **Model = Resnet9, Data = CIFAR100 data**
Val Accuracy=60%



In [ ]:
# %%writefile train_cifar.py
import time
import torch
import os
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torchsummary import summary

from model.dnn import DenseNeuralNet
from model.cnn import CNN_resnet9
from data.churn_data import ChurnDataset
from data.telescope_data import TelescopeDataset
from utils.util_functions import *
from data.mv_data import MVDataset
from tqdm.auto import trange, tqdm
from tqdm import trange
from torch.utils.data import Subset
from sklearn.model_selection import train_test_split


def get_cifar100_dataset(train = True):
  transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5071, 0.4865, 0.4409), std=(0.2673, 0.2564, 0.2762)),
  ])

  transform_test = transforms.Compose([
      transforms.ToTensor(),
      transforms.Normalize(mean=(0.5071, 0.4865, 0.4409), std=(0.2673, 0.2564, 0.2762)),
  ])

  if train == True:
      dataset = datasets.CIFAR100(root = './data', train = train, transform=transform_test, target_transform=None, download=True)
  else:
      dataset = datasets.CIFAR100(root = './data', train = train, transform=transform_test, target_transform=None, download=True)
  
  return dataset


def get_accuracy(logits, labels):
  preds = torch.argmax(logits, axis=1)
  matches = preds == labels
  return (matches.sum(), len(labels))


def evaluate(model, test_set, batch_size, criterion, ep = 0):
  test_loader = torch.utils.data.DataLoader(dataset = test_set, batch_size = batch_size, shuffle=True, num_workers=1)
  test_iterator = tqdm(test_loader, desc = 'Eval Iteration for epoch:'+str(ep+1), ncols = 900)
  device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
  
  model.eval()
  global_step = 0
  total_correct = 0
  total_samples = 0
  total_loss = 0.0
  with torch.no_grad():
    for step, inputs in enumerate(test_iterator):
      global_step +=1
      # if global_step > 500:
      #   break
      x, y = inputs[0].to(device), inputs[1].long().to(device)

      logits = model(x)
      loss = criterion(logits, y)
      correct, samples = get_accuracy(logits, y)
      total_correct +=correct.item()
      total_samples +=samples
      total_loss +=loss
  # print(total_correct, total_samples)
  acc = total_correct / total_samples
  total_loss = total_loss / global_step
  model.train()
  
  return (total_loss, acc)


max_lr = 0.01
grad_clip = 0.1
weight_decay = 1e-5

def train(model, train_set, val_set, test_set , batch_size = 16, learning_rate = 0.03, epochs = 5, eval_steps = 10, skip_train_set = True):
  # logging
  train_log = open("log/cifar_resnet9_train.log", "a")
  val_log = open("log/cifar_resnet9_val.log", "a")
  test_log = open("log/cifar_resnet9_test.log", "a")

  # GPU/CPU use
  device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
  print("Device: ", device)
  model = model.to(device)
  print("Model Summary:")
  summary(model, next(iter(train_set))[0].shape)

  # trainloader
  train_loader = torch.utils.data.DataLoader(dataset= train_set, batch_size=batch_size, shuffle=True, num_workers=1)
  
  # define loss & optimizer
  criterion = nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(model.parameters(), lr=max_lr, weight_decay=weight_decay)
  sched = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr, epochs=epochs, 
                                                steps_per_epoch=len(train_loader))
  # optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)

  # iterate over epoch
  global_step = 0
  for ep in tqdm(range(epochs), desc = ' Epoch Progress:', ncols=900):
    train_iterator = tqdm(train_loader, desc = 'Train Iteration for epoch:'+ str(ep+1), ncols=900)    
    running_loss = 0

    # iterate over batches
    for step, inputs in enumerate(train_iterator):
      model.train()
      global_step +=1
      optimizer.zero_grad()
      # predict, find loss, get grads, update weight
      x, y = inputs[0].to(device), inputs[1].to(device)
      logits = model(x)
      loss = criterion(logits, y)
      loss.backward()
      nn.utils.clip_grad_value_(model.parameters(), grad_clip)
      optimizer.step()
      sched.step()
      running_loss+=loss.item()

    # find validation accuracy
    val_loss, val_accuracy = evaluate(model, val_set, batch_size, criterion, ep)
    val_log.write("Epoch = {}, validation loss =  {}, validation accuracy = {} \n".format(ep+1, val_loss, val_accuracy))
    print("Step = %d, validation loss =  %.3f, validation accuracy = %.3f" %(global_step, val_loss, val_accuracy))
    
    # find train accuracy if needed
    if not skip_train_set:
      train_loss , train_accuracy = evaluate(model, train_set, batch_size, criterion, ep)
      train_log.write("Epoch = {}, training loss =  {}, training accuracy = {} \n".format(ep+1, train_loss, train_accuracy))
      print("Step = %d, training loss =  %.3f, training accuracy = %.3f" %(global_step, train_loss, train_accuracy))

  # find test accuracy with final model
  if test_set is not None:  
    test_loss, test_accuracy = evaluate(model, test_set, batch_size, criterion, ep)
    test_log.write("End of training, test loss =  {}, test accuracy = {} \n".format(test_loss, test_accuracy))
    print("End of Training, test loss =  %.3f, test accuracy = %.3f" %(test_loss, test_accuracy))

  # close log files
  train_log.close()
  val_log.close()
  test_log.close()

def main(load_model = False):
  ### config params
  output_classes = 100
  learning_rate = 0.001
  batch_size = 16
  epochs = 10
  eval_steps = 100
  model_dir = 'model_artifacts'
  model_name = 'cifar_resnet9_model.pt'
  criterion = nn.CrossEntropyLoss()
  ####

  train_set, val_set, test_set = None, None, None
  train_set = get_cifar100_dataset(train = True)
  val_set = get_cifar100_dataset(train = False)

  if load_model == True:
    model = torch.load(os.path.join(model_dir, model_name))
    val_loss, val_accuracy = evaluate(model, val_set, batch_size, criterion)
    print("Running evaluation on loaded model, validation loss = %f, validation accuracy = %f"%(val_loss, val_accuracy))

  else:
    model = CNN_resnet9(output_classes)
    train(model, train_set, val_set, test_set , batch_size = batch_size, learning_rate = learning_rate, epochs = epochs, eval_steps = eval_steps, skip_train_set = True)
    torch.save(model, os.path.join(model_dir, model_name))
  
  # Evaluate Quantization
  path_result = "data/results/"
  model_results = quantization_eval_results(model_name,train_set=train_set,test_set=val_set,batch_size=batch_size,criterion=criterion, method='all')
  print(model_results)
  model_results.to_csv(path_result + model_name[:-3]+'_v2.csv')

if __name__ == "__main__":
  main(load_model = True)


Files already downloaded and verified
Files already downloaded and verified



Running evaluation on loaded model, validation loss = 1.599501, validation accuracy = 0.597900



                    model       method  ...     test_loss test_acc
0  cifar_resnet9_model.pt  no rounding  ...  1.599501e+00   0.5979
1  cifar_resnet9_model.pt     mid-rise  ...  1.599263e+00   0.5980
2  cifar_resnet9_model.pt     mid-rise  ...  1.810148e+00   0.5631
3  cifar_resnet9_model.pt     mid-rise  ...  3.600675e+02   0.0101
4  cifar_resnet9_model.pt     mid-rise  ...  2.736699e+17   0.0090
5  cifar_resnet9_model.pt     mid-rise  ...  1.791628e+27   0.0093
6  cifar_resnet9_model.pt     mid-rise  ...  2.378884e+32   0.0097
7  cifar_resnet9_model.pt     mid-rise  ...           inf   0.0098

[8 rows x 8 columns]


# **Model = Resnet50, Data = CIFAR100 data**
Val Accuracy=50%

In [ ]:
# %%writefile train_cifar.py
import time
import torch
import os
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torchsummary import summary

from model.dnn import DenseNeuralNet
from model.cnn import CNN_resnet50
from data.churn_data import ChurnDataset
from data.telescope_data import TelescopeDataset
from utils.util_functions import *
from data.mv_data import MVDataset
from tqdm.auto import trange, tqdm
from tqdm import trange
from torch.utils.data import Subset
from sklearn.model_selection import train_test_split


def get_cifar100_dataset(train = True):
  transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
  ])

  transform_test = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
      transforms.ToTensor(),
      transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
  ])

  if train == True:
      dataset = datasets.CIFAR100(root = './data', train = train, transform=transform_test, target_transform=None, download=True)
  else:
      dataset = datasets.CIFAR100(root = './data', train = train, transform=transform_test, target_transform=None, download=True)
  
  return dataset


def get_accuracy(logits, labels):
  preds = torch.argmax(logits, axis=1)
  matches = preds == labels
  return (matches.sum(), len(labels))


def evaluate(model, test_set, batch_size, criterion, ep = 0):
  test_loader = torch.utils.data.DataLoader(dataset = test_set, batch_size = batch_size, shuffle=True, num_workers=1)
  test_iterator = tqdm(test_loader, desc = 'Eval Iteration for epoch:'+str(ep+1), ncols = 900)
  device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
  
  model.eval()
  global_step = 0
  total_correct = 0
  total_samples = 0
  total_loss = 0.0
  with torch.no_grad():
    for step, inputs in enumerate(test_iterator):
      global_step +=1
      # if global_step > 500:
      #   break
      x, y = inputs[0].to(device), inputs[1].long().to(device)

      logits = model(x)
      loss = criterion(logits, y)
      correct, samples = get_accuracy(logits, y)
      total_correct +=correct.item()
      total_samples +=samples
      total_loss +=loss
  # print(total_correct, total_samples)
  acc = total_correct / total_samples
  total_loss = total_loss / global_step
  model.train()
  
  return (total_loss, acc)


def train(model, train_set, val_set, test_set , batch_size = 16, learning_rate = 0.03, epochs = 5, eval_steps = 10, skip_train_set = True):
  # logging
  train_log = open("log/cifar_resnet50_train.log", "a")
  val_log = open("log/cifar_resnet50_val.log", "a")
  test_log = open("log/cifar_resnet50_test.log", "a")

  # GPU/CPU use
  device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
  print("Device: ", device)
  model = model.to(device)
  print("Model Summary:")
  summary(model, next(iter(train_set))[0].shape)
  
  # define loss & optimizer
  criterion = nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
  # optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)

  # iterate over epoch
  train_loader = torch.utils.data.DataLoader(dataset= train_set, batch_size=batch_size, shuffle=True, num_workers=1)
  global_step = 0
  for ep in tqdm(range(epochs), desc = ' Epoch Progress:', ncols=900):
    train_iterator = tqdm(train_loader, desc = 'Train Iteration for epoch:'+ str(ep+1), ncols=900)    
    running_loss = 0

    # iterate over batches
    for step, inputs in enumerate(train_iterator):
      model.train()
      global_step +=1
      optimizer.zero_grad()
      # predict, find loss, get grads, update weight
      x, y = inputs[0].to(device), inputs[1].to(device)
      logits = model(x)
      loss = criterion(logits, y)
      loss.backward()
      optimizer.step()
      running_loss+=loss.item()

    # find validation accuracy
    val_loss, val_accuracy = evaluate(model, val_set, batch_size, criterion, ep)
    val_log.write("Epoch = {}, validation loss =  {}, validation accuracy = {} \n".format(ep+1, val_loss, val_accuracy))
    print("Step = %d, validation loss =  %.3f, validation accuracy = %.3f" %(global_step, val_loss, val_accuracy))
    
    # find train accuracy if needed
    if not skip_train_set:
      train_loss , train_accuracy = evaluate(model, train_set, batch_size, criterion, ep)
      train_log.write("Epoch = {}, training loss =  {}, training accuracy = {} \n".format(ep+1, train_loss, train_accuracy))
      print("Step = %d, training loss =  %.3f, training accuracy = %.3f" %(global_step, train_loss, train_accuracy))

  # find test accuracy with final model
  if test_set is not None:  
    test_loss, test_accuracy = evaluate(model, test_set, batch_size, criterion, ep)
    test_log.write("End of training, test loss =  {}, test accuracy = {} \n".format(test_loss, test_accuracy))
    print("End of Training, test loss =  %.3f, test accuracy = %.3f" %(test_loss, test_accuracy))

  # close log files
  train_log.close()
  val_log.close()
  test_log.close()

def main(load_model = False):
  ### config params
  output_classes = 100
  learning_rate = 0.0009
  batch_size = 16
  epochs = 10
  eval_steps = 100
  model_dir = 'model_artifacts'
  model_name = 'cifar_resnet50_model.pt'
  criterion = nn.CrossEntropyLoss()
  ####

  train_set, val_set, test_set = None, None, None
  train_set = get_cifar100_dataset(train = True)
  val_set = get_cifar100_dataset(train = False)

  if load_model == True:
    model = torch.load(os.path.join(model_dir, model_name))
    val_loss, val_accuracy = evaluate(model, val_set, batch_size, criterion)
    print("Running evaluation on loaded model, validation loss = %f, validation accuracy = %f"%(val_loss, val_accuracy))

  else:
    model = CNN_resnet50(output_classes)
    train(model, train_set, val_set, test_set , batch_size = batch_size, learning_rate = learning_rate, epochs = epochs, eval_steps = eval_steps, skip_train_set = True)
    torch.save(model, os.path.join(model_dir, model_name))

  # Evaluate Quantization
  path_result = "data/results/"
  model_results = quantization_eval_results(model_name,train_set=train_set,test_set=val_set,batch_size=batch_size,criterion=criterion, method='mid-rise')
  print(model_results)
  model_results.to_csv(path_result + model_name[:-3] +'_onlyMidrise.csv')


if __name__ == "__main__":
  main(load_model = True)


Files already downloaded and verified
Files already downloaded and verified



Running evaluation on loaded model, validation loss = 1.780087, validation accuracy = 0.502900



                     model       method  ...     test_loss test_acc
0  cifar_resnet50_model.pt  no rounding  ...  1.780087e+00   0.5029
1  cifar_resnet50_model.pt     mid-rise  ...  1.780087e+00   0.5032
2  cifar_resnet50_model.pt     mid-rise  ...  1.781369e+00   0.5037
3  cifar_resnet50_model.pt     mid-rise  ...  2.028440e+00   0.4634
4  cifar_resnet50_model.pt     mid-rise  ...  1.233747e+03   0.0093
5  cifar_resnet50_model.pt     mid-rise  ...  2.271708e+20   0.0100
6  cifar_resnet50_model.pt     mid-rise  ...           NaN   0.0104
7  cifar_resnet50_model.pt     mid-rise  ...           NaN   0.0100

[8 rows x 8 columns]


# **Model = VGG16, Data= CIFAR100**
Val Accuracy=19%

In [ ]:
# %%writefile train_cifar.py
import time
import torch
import os
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torchsummary import summary

from model.dnn import DenseNeuralNet
from model.cnn import CNN_vgg16
from data.churn_data import ChurnDataset
from data.telescope_data import TelescopeDataset
from utils.util_functions import *
from data.mv_data import MVDataset
from tqdm.auto import trange, tqdm
from tqdm import trange
from torch.utils.data import Subset
from sklearn.model_selection import train_test_split


def get_cifar100_dataset(train = True):
  transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
  ])

  transform_test = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
      transforms.ToTensor(),
      transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
  ])

  if train == True:
      dataset = datasets.CIFAR100(root = './data', train = train, transform=transform_test, target_transform=None, download=True)
  else:
      dataset = datasets.CIFAR100(root = './data', train = train, transform=transform_test, target_transform=None, download=True)
  
  return dataset


def get_accuracy(logits, labels):
  preds = torch.argmax(logits, axis=1)
  matches = preds == labels
  return (matches.sum(), len(labels))


def evaluate(model, test_set, batch_size, criterion, ep = 0):
  test_loader = torch.utils.data.DataLoader(dataset = test_set, batch_size = batch_size, shuffle=True, num_workers=1)
  test_iterator = tqdm(test_loader, desc = 'Eval Iteration for epoch:'+str(ep+1), ncols = 900)
  device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
  
  model.eval()
  global_step = 0
  total_correct = 0
  total_samples = 0
  total_loss = 0.0
  with torch.no_grad():
    for step, inputs in enumerate(test_iterator):
      global_step +=1
      # if global_step > 500:
      #   break
      x, y = inputs[0].to(device), inputs[1].long().to(device)

      logits = model(x)
      loss = criterion(logits, y)
      correct, samples = get_accuracy(logits, y)
      total_correct +=correct.item()
      total_samples +=samples
      total_loss +=loss
  # print(total_correct, total_samples)
  acc = total_correct / total_samples
  total_loss = total_loss / global_step
  model.train()
  
  return (total_loss, acc)


def train(model, train_set, val_set, test_set , batch_size = 16, learning_rate = 0.03, epochs = 5, eval_steps = 10, skip_train_set = True):
  # logging
  train_log = open("log/cifar_vgg16_train.log", "a")
  val_log = open("log/cifar_vgg16_val.log", "a")
  test_log = open("log/cifar_vgg16_test.log", "a")

  # GPU/CPU use
  device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
  print("Device: ", device)
  model = model.to(device)
  print("Model Summary:")
  summary(model, next(iter(train_set))[0].shape)
  
  # define loss & optimizer
  criterion = nn.CrossEntropyLoss()
  # optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
  optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay=1e-6, momentum=0.9, nesterov=True)

  # iterate over epoch
  train_loader = torch.utils.data.DataLoader(dataset= train_set, batch_size=batch_size, shuffle=True, num_workers=1)
  global_step = 0
  for ep in tqdm(range(epochs), desc = ' Epoch Progress:', ncols=900):
    train_iterator = tqdm(train_loader, desc = 'Train Iteration for epoch:'+ str(ep+1), ncols=900)    
    running_loss = 0

    # iterate over batches
    for step, inputs in enumerate(train_iterator):
      model.train()
      global_step +=1
      optimizer.zero_grad()
      # predict, find loss, get grads, update weight
      x, y = inputs[0].to(device), inputs[1].to(device)
      logits = model(x)
      loss = criterion(logits, y)
      loss.backward()
      optimizer.step()
      running_loss+=loss.item()

    # find validation accuracy
    val_loss, val_accuracy = evaluate(model, val_set, batch_size, criterion, ep)
    val_log.write("Epoch = {}, validation loss =  {}, validation accuracy = {} \n".format(ep+1, val_loss, val_accuracy))
    print("Step = %d, validation loss =  %.3f, validation accuracy = %.3f" %(global_step, val_loss, val_accuracy))
    
    # find train accuracy if needed
    if not skip_train_set:
      train_loss , train_accuracy = evaluate(model, train_set, batch_size, criterion, ep)
      train_log.write("Epoch = {}, training loss =  {}, training accuracy = {} \n".format(ep+1, train_loss, train_accuracy))
      print("Step = %d, training loss =  %.3f, training accuracy = %.3f" %(global_step, train_loss, train_accuracy))

  # find test accuracy with final model
  if test_set is not None:  
    test_loss, test_accuracy = evaluate(model, test_set, batch_size, criterion, ep)
    test_log.write("End of training, test loss =  {}, test accuracy = {} \n".format(test_loss, test_accuracy))
    print("End of Training, test loss =  %.3f, test accuracy = %.3f" %(test_loss, test_accuracy))

  # close log files
  train_log.close()
  val_log.close()
  test_log.close()

def main(load_model = False):
  ### config params
  output_classes = 100
  learning_rate = 0.01
  batch_size = 32
  epochs = 10
  eval_steps = 100
  model_dir = 'model_artifacts'
  model_name = 'cifar_vgg16_model.pt'
  criterion = nn.CrossEntropyLoss()
  ####

  train_set, val_set, test_set = None, None, None
  train_set = get_cifar100_dataset(train = True)
  val_set = get_cifar100_dataset(train = False)

  if load_model == True:
    model = torch.load(os.path.join(model_dir, model_name))
    val_loss, val_accuracy = evaluate(model, val_set, batch_size, criterion)
    print("Running evaluation on loaded model, validation loss = %f, validation accuracy = %f"%(val_loss, val_accuracy))

  else:
    model = CNN_vgg16(output_classes)
    train(model, train_set, val_set, test_set , batch_size = batch_size, learning_rate = learning_rate, epochs = epochs, eval_steps = eval_steps, skip_train_set = True)
    torch.save(model, os.path.join(model_dir, model_name))

  # Evaluate Quantization
  path_result = "data/results/"
  model_results = quantization_eval_results(model_name,train_set=train_set,test_set=val_set,batch_size=batch_size,criterion=criterion)
  print(model_results)
  model_results.to_csv(path_result + model_name[:-3] +".csv")


if __name__ == "__main__":
  main(load_model = False)


Files already downloaded and verified
Files already downloaded and verified
Device:  cuda:0
Model Summary:
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 224, 224]           1,792
              ReLU-2         [-1, 64, 224, 224]               0
            Conv2d-3         [-1, 64, 224, 224]          36,928
              ReLU-4         [-1, 64, 224, 224]               0
         MaxPool2d-5         [-1, 64, 112, 112]               0
            Conv2d-6        [-1, 128, 112, 112]          73,856
              ReLU-7        [-1, 128, 112, 112]               0
            Conv2d-8        [-1, 128, 112, 112]         147,584
              ReLU-9        [-1, 128, 112, 112]               0
        MaxPool2d-10          [-1, 128, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]         295,168
             ReLU-12          [-1, 256, 56, 56]             


Step = 1563, validation loss =  4.579, validation accuracy = 0.044



Step = 3126, validation loss =  4.491, validation accuracy = 0.135



Step = 4689, validation loss =  4.461, validation accuracy = 0.164



Step = 6252, validation loss =  4.451, validation accuracy = 0.172


...


Step = 9378, validation loss =  4.444, validation accuracy = 0.178



Step = 10941, validation loss =  4.442, validation accuracy = 0.181



Step = 12504, validation loss =  4.434, validation accuracy = 0.189



Step = 14067, validation loss =  4.431, validation accuracy = 0.192



Step = 15630, validation loss =  4.429, validation accuracy = 0.193

